In [3]:
import selenium
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import os
import pyarrow
from selenium import webdriver
from bs4 import BeautifulSoup
import time


pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns', None)
warnings.filterwarnings('ignore')

# Data Extraction

Financial 10-K report were extracted from SEC's EDGAR db.

`Microsoft`: 'https://www.sec.gov/edgar/browse/?CIK=789019&owner=exclude'

`Tesla`:'https://www.sec.gov/edgar/browse/?CIK=1318605&owner=exclude'

`Apple`:'https://www.sec.gov/edgar/browse/?CIK=320193&owner=exclude'


In [6]:
# load the datasets for each company

microsoft_data = pd.read_excel('../dataset/raw/Extracted_report/Microsoft 10K_Report.xlsx')

apple_data = pd.read_excel('../dataset/raw/Extracted_report/Apple 10K_Report.xlsx')

tesla_data = pd.read_excel('../dataset/raw/Extracted_report/Tesla 10K_Report.xlsx')

print('dataset ingested')


dataset ingested


### Data Cleaning


In [7]:
# Cleaning the data for Microsoft 10k report
microsoft_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   Microsoft Corporation   Consolidated Balance Sheets
(in millions, except per share data)  0 non-null      float64
 1   year                                                                                      5 non-null      object 
 2   2024                                                                                      5 non-null      int64  
 3   2023                                                                                      5 non-null      int64  
 4   2022                                                                                      5 non-null      int64  
 5   2021                                                          

In [8]:
#dropping a column.
microsoft_data.drop(columns=["Microsoft Corporation   Consolidated Balance Sheets\n(in millions, except per share data)"],inplace=True)

print("column dropped successfully")

column dropped successfully


In [9]:
#reshaping the dataframe by Transposing it.
transpose_df = microsoft_data.T
transpose_df 

,0,1,2,3,4
year,total_revenue,net_income,total_assets,total_liabilities,cash_flow_from_operating_activities
2024,245122,88136,512163,243686,118548
2023,211915,72361,411976,205753,87582
2022,198270,72738,364840,198298,89035
2021,168088,61271,333779,191791,76740
2020,143015,44281,301311,183007,60675


In [10]:
transpose_df.columns

RangeIndex(start=0, stop=5, step=1)

In [11]:
transpose_df.columns = transpose_df.columns.astype(str)

transpose_df.reset_index(drop=False, inplace=True)

columns_rename = {
    "0":'Total_revenue',
    "1":'Net_income',
    "2":'Total_assets',
    "3":'Total_liabilities',
    "4":'Cash_flow_from_operating_activities',
}
transpose_df.rename(columns=columns_rename, inplace=True)

transpose_df


,index,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities
0,year,total_revenue,net_income,total_assets,total_liabilities,cash_flow_from_operating_activities
1,2024,245122,88136,512163,243686,118548
2,2023,211915,72361,411976,205753,87582
3,2022,198270,72738,364840,198298,89035
4,2021,168088,61271,333779,191791,76740
5,2020,143015,44281,301311,183007,60675


In [12]:
transpose_df.rename(columns={'index':'Year'}, inplace=True)

transpose_df.drop(index=[0], inplace=True)

transpose_df

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities
1,2024,245122,88136,512163,243686,118548
2,2023,211915,72361,411976,205753,87582
3,2022,198270,72738,364840,198298,89035
4,2021,168088,61271,333779,191791,76740
5,2020,143015,44281,301311,183007,60675


In [13]:
transpose_df['Company'] = 'Microsoft'

transpose_df.head()

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities,Company
1,2024,245122,88136,512163,243686,118548,Microsoft
2,2023,211915,72361,411976,205753,87582,Microsoft
3,2022,198270,72738,364840,198298,89035,Microsoft
4,2021,168088,61271,333779,191791,76740,Microsoft
5,2020,143015,44281,301311,183007,60675,Microsoft


In [15]:
#Save the clean the dataset

transpose_df.to_csv('../dataset/processed/10k_reports/Microsoft/Microsoft 10K_Report.csv')

In [16]:
# Cleaning the data for Tesla 10k report
tesla_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   Tesla, Inc.
Consolidated Balance Sheets
(in millions, except per share data)  0 non-null      float64
 1   year                                                                          5 non-null      object 
 2   2024                                                                          0 non-null      float64
 3   2023                                                                          5 non-null      int64  
 4   2022                                                                          5 non-null      int64  
 5   2021                                                                          5 non-null      int64  
 6   2020                                   

In [17]:
tesla_data.columns

Index(['Tesla, Inc.\nConsolidated Balance Sheets\n(in millions, except per share data)',
                                                                                 'year',
                                                                                   2024,
                                                                                   2023,
                                                                                   2022,
                                                                                   2021,
                                                                                   2020],
      dtype='object')

In [18]:
#dropping a column.
tesla_data.drop(columns=["Tesla, Inc.\nConsolidated Balance Sheets\n(in millions, except per share data)"], inplace=True)

print(f"column dropped successfully.. {tesla_data.columns}")

column dropped successfully.. Index(['year', 2024, 2023, 2022, 2021, 2020], dtype='object')


In [19]:
tesla_data.head()

,year,2024,2023,2022,2021,2020
0,total_revenue,NaN,96773,81462,53823,31536
1,net_income,NaN,14974,12587,5644,862
2,total_assets,NaN,106618,82338,62131,52148
3,total_liabilities,NaN,43009,36440,30548,28418
4,cash_flow_from_operating_activities,NaN,13256,14724,11497,5943


In [20]:
#reshaping the dataframe by Transposing it.
transpose_df = tesla_data.T
transpose_df 

,0,1,2,3,4
year,total_revenue,net_income,total_assets,total_liabilities,cash_flow_from_operating_activities
2024,NaN,NaN,NaN,NaN,NaN
2023,96773,14974,106618,43009,13256
2022,81462,12587,82338,36440,14724
2021,53823,5644,62131,30548,11497
2020,31536,862,52148,28418,5943


In [21]:
transpose_df.columns = transpose_df.columns.astype(str)

transpose_df.reset_index(drop=False, inplace=True)

columns_rename = {
    "0":'Total_revenue',
    "1":'Net_income',
    "2":'Total_assets',
    "3":'Total_liabilities',
    "4":'Cash_flow_from_operating_activities',
}
transpose_df.rename(columns=columns_rename, inplace=True)

transpose_df


,index,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities
0,year,total_revenue,net_income,total_assets,total_liabilities,cash_flow_from_operating_activities
1,2024,NaN,NaN,NaN,NaN,NaN
2,2023,96773,14974,106618,43009,13256
3,2022,81462,12587,82338,36440,14724
4,2021,53823,5644,62131,30548,11497
5,2020,31536,862,52148,28418,5943


In [22]:
transpose_df.rename(columns={'index':'Year'}, inplace=True)

transpose_df.drop(index=[0], inplace=True)

transpose_df

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities
1,2024,NaN,NaN,NaN,NaN,NaN
2,2023,96773,14974,106618,43009,13256
3,2022,81462,12587,82338,36440,14724
4,2021,53823,5644,62131,30548,11497
5,2020,31536,862,52148,28418,5943


In [23]:
#check for missing values
transpose_df.isna().sum()

Year                                   0
Total_revenue                          1
Net_income                             1
Total_assets                           1
Total_liabilities                      1
Cash_flow_from_operating_activities    1
dtype: int64

In [24]:
#Fill NaN values with mean
transpose_df = transpose_df.fillna(transpose_df.mean())

transpose_df

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities
1,2024,65898.5,8516.75,75808.75,34603.75,11355.0
2,2023,96773.0,14974.00,106618.00,43009.00,13256.0
3,2022,81462.0,12587.00,82338.00,36440.00,14724.0
4,2021,53823.0,5644.00,62131.00,30548.00,11497.0
5,2020,31536.0,862.00,52148.00,28418.00,5943.0


In [25]:
transpose_df['Company'] = 'Tesla'

transpose_df.head()

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities,Company
1,2024,65898.5,8516.75,75808.75,34603.75,11355.0,Tesla
2,2023,96773.0,14974.00,106618.00,43009.00,13256.0,Tesla
3,2022,81462.0,12587.00,82338.00,36440.00,14724.0,Tesla
4,2021,53823.0,5644.00,62131.00,30548.00,11497.0,Tesla
5,2020,31536.0,862.00,52148.00,28418.00,5943.0,Tesla


In [26]:
#Save the clean dataframe to tesla 10k report

transpose_df.to_csv('../dataset/processed/10k_reports/Tesla/Tesla 10K_Report.csv')

In [27]:
# Cleaning the data for Apple 10k report
apple_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column                                                                                                                                            Non-Null Count  Dtype  
---  ------                                                                                                                                            --------------  -----  
 0   Apple Inc.
CONSOLIDATED STATEMENTS OF OPERATIONS
(In millions, except number of shares, which are reflected in thousands, and per-share amounts)  0 non-null      float64
 1   year                                                                                                                                              5 non-null      object 
 2   2024                                                                                                                                              5 non-null      int64  
 3   2023                

In [28]:
apple_data.columns

Index(['Apple Inc.\nCONSOLIDATED STATEMENTS OF OPERATIONS\n(In millions, except number of shares, which are reflected in thousands, and per-share amounts)',
                                                                                                                                                     'year',
                                                                                                                                                       2024,
                                                                                                                                                       2023,
                                                                                                                                                       2022,
                                                                                                                                                       2021,
                                                          

In [29]:
#dropping a column.
apple_data.drop(columns=["Apple Inc.\nCONSOLIDATED STATEMENTS OF OPERATIONS\n(In millions, except number of shares, which are reflected in thousands, and per-share amounts)"], inplace=True)

print(f"column dropped successfully.. {apple_data.columns}")

column dropped successfully.. Index(['year', 2024, 2023, 2022, 2021, 2020], dtype='object')


In [30]:
apple_data.head()

,year,2024,2023,2022,2021,2020
0,total_revenue,391035,383285,394328,365817,274515
1,net_income,93736,96995,99803,94680,57411
2,total_assets,364980,352583,352755,351002,323888
3,total_liabilities,308030,290437,302083,287912,258549
4,cash_flow_from_operating_activities,118254,110543,122151,104038,80674


In [31]:
#reshaping the dataframe by Transposing it.
transpose_df = apple_data.T
transpose_df 

,0,1,2,3,4
year,total_revenue,net_income,total_assets,total_liabilities,cash_flow_from_operating_activities
2024,391035,93736,364980,308030,118254
2023,383285,96995,352583,290437,110543
2022,394328,99803,352755,302083,122151
2021,365817,94680,351002,287912,104038
2020,274515,57411,323888,258549,80674


In [32]:
transpose_df.columns = transpose_df.columns.astype(str)

transpose_df.reset_index(drop=False, inplace=True)

columns_rename = {
    "0":'Total_revenue',
    "1":'Net_income',
    "2":'Total_assets',
    "3":'Total_liabilities',
    "4":'Cash_flow_from_operating_activities',
}
transpose_df.rename(columns=columns_rename, inplace=True)

transpose_df


,index,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities
0,year,total_revenue,net_income,total_assets,total_liabilities,cash_flow_from_operating_activities
1,2024,391035,93736,364980,308030,118254
2,2023,383285,96995,352583,290437,110543
3,2022,394328,99803,352755,302083,122151
4,2021,365817,94680,351002,287912,104038
5,2020,274515,57411,323888,258549,80674


In [33]:
transpose_df.rename(columns={'index':'Year'}, inplace=True)

transpose_df.drop(index=[0], inplace=True)

transpose_df

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities
1,2024,391035,93736,364980,308030,118254
2,2023,383285,96995,352583,290437,110543
3,2022,394328,99803,352755,302083,122151
4,2021,365817,94680,351002,287912,104038
5,2020,274515,57411,323888,258549,80674


In [34]:
transpose_df['Company'] = 'Apple'

transpose_df.head()

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities,Company
1,2024,391035,93736,364980,308030,118254,Apple
2,2023,383285,96995,352583,290437,110543,Apple
3,2022,394328,99803,352755,302083,122151,Apple
4,2021,365817,94680,351002,287912,104038,Apple
5,2020,274515,57411,323888,258549,80674,Apple


In [35]:
#Save the clean dataframe to tesla 10k report

transpose_df.to_csv('../dataset/processed/10k_reports/Apple/Apple 10K_Report.csv')

In [37]:
#Combining the reports into one dataframe

import os, sys
sys.path.append('../src/data')

from DataMerge import load_and_merge_datasets

file_paths = [
    r"..\dataset\processed\10k_reports\Apple\Apple 10K_Report.csv",
    r"..\dataset\processed\10k_reports\Microsoft\Microsoft 10K_Report.csv",
    r"..\dataset\processed\10k_reports\Tesla\Tesla 10K_Report.csv",
]

merged_df = load_and_merge_datasets(file_paths)


merged_df.head(10)

,Unnamed: 0,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities,Company
0,1,2024,391035.0,93736.0,364980.0,308030.0,118254.0,Apple
1,2,2023,383285.0,96995.0,352583.0,290437.0,110543.0,Apple
2,3,2022,394328.0,99803.0,352755.0,302083.0,122151.0,Apple
3,4,2021,365817.0,94680.0,351002.0,287912.0,104038.0,Apple
4,5,2020,274515.0,57411.0,323888.0,258549.0,80674.0,Apple
5,1,2024,245122.0,88136.0,512163.0,243686.0,118548.0,Microsoft
6,2,2023,211915.0,72361.0,411976.0,205753.0,87582.0,Microsoft
7,3,2022,198270.0,72738.0,364840.0,198298.0,89035.0,Microsoft
8,4,2021,168088.0,61271.0,333779.0,191791.0,76740.0,Microsoft
9,5,2020,143015.0,44281.0,301311.0,183007.0,60675.0,Microsoft


In [38]:
#dropping a column.
merged_df.drop(columns=['Unnamed: 0'], inplace=True)

merged_df.reset_index(drop=True, inplace=True)

In [39]:
merged_df

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities,Company
0,2024,391035.0,93736.00,364980.00,308030.00,118254.0,Apple
1,2023,383285.0,96995.00,352583.00,290437.00,110543.0,Apple
2,2022,394328.0,99803.00,352755.00,302083.00,122151.0,Apple
3,2021,365817.0,94680.00,351002.00,287912.00,104038.0,Apple
4,2020,274515.0,57411.00,323888.00,258549.00,80674.0,Apple
5,2024,245122.0,88136.00,512163.00,243686.00,118548.0,Microsoft
6,2023,211915.0,72361.00,411976.00,205753.00,87582.0,Microsoft
7,2022,198270.0,72738.00,364840.00,198298.00,89035.0,Microsoft
8,2021,168088.0,61271.00,333779.00,191791.00,76740.0,Microsoft
9,2020,143015.0,44281.00,301311.00,183007.00,60675.0,Microsoft


In [40]:
#Save the final report
merged_df.to_csv("../dataset/Final_report/10k_final_report.csv", index = False)
print("Merged File saved successfully..")

Merged File saved successfully..


### Data Manipulation

In [41]:
df = pd.read_csv("../dataset/Final_report/10k_final_report.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 15 non-null     int64  
 1   Total_revenue                        15 non-null     float64
 2   Net_income                           15 non-null     float64
 3   Total_assets                         15 non-null     float64
 4   Total_liabilities                    15 non-null     float64
 5   Cash_flow_from_operating_activities  15 non-null     float64
 6   Company                              15 non-null     object 
dtypes: float64(5), int64(1), object(1)
memory usage: 972.0+ bytes


**Calculating Year-by-Year growth rates for Total Revenue and Net Income**

In [42]:
df['Revenue_growth_(%)'] = df.groupby('Company')['Total_revenue'].pct_change() * 100

df['Net_income_growth_(%)'] = df.groupby('Company')['Net_income'].pct_change() * 100

df.head()

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities,Company,Revenue_growth_(%),Net_income_growth_(%)
0,2024,391035.0,93736.0,364980.0,308030.0,118254.0,Apple,NaN,NaN
1,2023,383285.0,96995.0,352583.0,290437.0,110543.0,Apple,-1.981920,3.476786
2,2022,394328.0,99803.0,352755.0,302083.0,122151.0,Apple,2.881146,2.894995
3,2021,365817.0,94680.0,351002.0,287912.0,104038.0,Apple,-7.230275,-5.133112
4,2020,274515.0,57411.0,323888.0,258549.0,80674.0,Apple,-24.958381,-39.363118


**Calculating Year-by-Year growth rates for Total Assets, Total Liabilities and Cash flow from Operations Activities**

In [43]:
df['Assets_growth_(%)'] = df.groupby('Company')['Total_assets'].pct_change() * 100

df['Liabilities_growth_(%)'] = df.groupby('Company')['Total_liabilities'].pct_change() * 100

df['Cash_flow_from_operating_growth_(%)'] = df.groupby('Company')['Cash_flow_from_operating_activities'].pct_change() * 100

In [44]:
#Check for nan value
df.isna().sum()

Year                                   0
Total_revenue                          0
Net_income                             0
Total_assets                           0
Total_liabilities                      0
Cash_flow_from_operating_activities    0
Company                                0
Revenue_growth_(%)                     3
Net_income_growth_(%)                  3
Assets_growth_(%)                      3
Liabilities_growth_(%)                 3
Cash_flow_from_operating_growth_(%)    3
dtype: int64

In [45]:
#Filling the NaN values in the dataset with the mean.
num_col = df.select_dtypes(include=['number']).columns

df[num_col] = df[num_col].fillna(df[num_col].mean())

df.head()

,Year,Total_revenue,Net_income,Total_assets,Total_liabilities,Cash_flow_from_operating_activities,Company,Revenue_growth_(%),Net_income_growth_(%),Assets_growth_(%),Liabilities_growth_(%),Cash_flow_from_operating_growth_(%)
0,2024,391035.0,93736.0,364980.0,308030.0,118254.0,Apple,-10.476779,-14.917162,-6.962902,-4.813966,-11.243334
1,2023,383285.0,96995.0,352583.0,290437.0,110543.0,Apple,-1.981920,3.476786,-3.396624,-5.711457,-6.520710
2,2022,394328.0,99803.0,352755.0,302083.0,122151.0,Apple,2.881146,2.894995,0.048783,4.009820,10.500891
3,2021,365817.0,94680.0,351002.0,287912.0,104038.0,Apple,-7.230275,-5.133112,-0.496945,-4.691095,-14.828368
4,2020,274515.0,57411.0,323888.0,258549.0,80674.0,Apple,-24.958381,-39.363118,-7.724742,-10.198602,-22.457179


In [46]:
#Check for nan value
df.isna().sum()

Year                                   0
Total_revenue                          0
Net_income                             0
Total_assets                           0
Total_liabilities                      0
Cash_flow_from_operating_activities    0
Company                                0
Revenue_growth_(%)                     0
Net_income_growth_(%)                  0
Assets_growth_(%)                      0
Liabilities_growth_(%)                 0
Cash_flow_from_operating_growth_(%)    0
dtype: int64

In [47]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,15.0,2022.000000,1.463850,2020.000000,2021.000000,2022.000000,2023.000000,2024.000000
Total_revenue,15.0,206992.166667,130671.419901,31536.000000,89117.500000,198270.000000,320166.000000,394328.000000
Net_income,15.0,54933.050000,37462.787284,862.000000,13780.500000,61271.000000,90936.000000,99803.000000
Total_assets,15.0,269888.050000,150068.418679,52148.000000,94478.000000,333779.000000,358797.500000,512163.000000
Total_liabilities,15.0,176170.983333,110879.080126,28418.000000,39724.500000,198298.000000,273230.500000,308030.000000
Cash_flow_from_operating_activities,15.0,68334.333333,44979.373588,5943.000000,13990.000000,80674.000000,107290.500000,122151.000000
Revenue_growth_(%),15.0,-10.476779,19.567940,-41.407948,-15.522119,-10.476779,-6.834589,46.851598
Net_income_growth_(%),15.0,-14.917162,34.317763,-84.727144,-22.813872,-14.917162,-2.306057,75.818241
Assets_growth_(%),15.0,-6.962902,15.121106,-24.541524,-13.754553,-7.724742,-5.179763,40.640757
Liabilities_growth_(%),15.0,-4.813966,9.699595,-16.169045,-8.585618,-4.813966,-4.101631,24.289997


In [48]:
#Save the final report
df.to_json("../dataset/Final_report/10k_final_report.json",)
print("Final report File saved successfully..")

Final report File saved successfully..


In [49]:
# summarizing my findings

summary = df.groupby('Company').agg({
    'Revenue_growth_(%)':'mean',
    'Net_income_growth_(%)':'mean',
    'Assets_growth_(%)':'mean',
    'Liabilities_growth_(%)':'mean',
    'Cash_flow_from_operating_growth_(%)':'mean',
    
}).reset_index()

summary

,Company,Revenue_growth_(%),Net_income_growth_(%),Assets_growth_(%),Liabilities_growth_(%),Cash_flow_from_operating_growth_(%)
0,Apple,-8.353242,-10.608322,-3.706486,-4.281060,-8.909740
1,Microsoft,-12.120416,-15.157741,-11.241376,-6.372999,-14.089776
2,Tesla,-10.956679,-18.985423,-5.940845,-3.787839,-10.730487


In [50]:
print(f"Overall Growth or Loss rate for Apple, Microsoft and Tesla from Fiscal Year 2020 - 2024:")
summary

Overall Growth or Loss rate for Apple, Microsoft and Tesla from Fiscal Year 2020 - 2024:


,Company,Revenue_growth_(%),Net_income_growth_(%),Assets_growth_(%),Liabilities_growth_(%),Cash_flow_from_operating_growth_(%)
0,Apple,-8.353242,-10.608322,-3.706486,-4.281060,-8.909740
1,Microsoft,-12.120416,-15.157741,-11.241376,-6.372999,-14.089776
2,Tesla,-10.956679,-18.985423,-5.940845,-3.787839,-10.730487


In [51]:
#Save the summary of the 10-K report
summary.to_json("../dataset/Final_report/10k_summary_report.json",)


In [56]:
df[['Revenue_growth_(%)','Year','Company']]

,Revenue_growth_(%),Year,Company
0,-10.476779,2024,Apple
1,-1.981920,2023,Apple
2,2.881146,2022,Apple
3,-7.230275,2021,Apple
4,-24.958381,2020,Apple
5,-10.476779,2024,Microsoft
6,-13.547132,2023,Microsoft
7,-6.438902,2022,Microsoft
8,-15.222676,2021,Microsoft
9,-14.916591,2020,Microsoft
